<a href="https://colab.research.google.com/github/yoneken1/colab_tensorflow/blob/master/tensorflow_batch_normalization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NGな実装


---



1. tf.keras.batchnorm　＆　カスタムestimator

In [0]:
import tensorflow as tf

def model_fn(features, labels, mode):
  
  training = (mode == tf.estimator.ModeKeys.TRAIN)
  x = tf.keras.layers.BatchNormalization()(features,training=training)
  y = tf.keras.layers.Dense(1)(x)
  predictions = {
      "prob": y,
  }
  
  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  loss = tf.reduce_mean(tf.losses.mean_squared_error(labels,y))

  if mode == tf.estimator.ModeKeys.TRAIN:
      optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
     
      update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
      with tf.control_dependencies(update_ops):
        train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())

      return tf.estimator.EstimatorSpec(
          mode=tf.estimator.ModeKeys.TRAIN,
          loss=loss,
          train_op=train_op)
    
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss)
  
estimator = tf.estimator.Estimator(
    model_fn=model_fn)

input_fn = lambda:(tf.constant([[0], [1], [2], [3]], dtype=tf.float32),tf.constant([[0], [-1], [-2], [-3]], dtype=tf.float32))

estimator.train(input_fn,steps=5000)
result = estimator.evaluate(input_fn,steps=4)

# OKな実装


---



tf.layers.BatchNormalziation　＆　カスタムestimator

In [0]:
import tensorflow as tf

def model_fn(features, labels, mode):
  
  training = (mode == tf.estimator.ModeKeys.TRAIN)
  x = tf.layers.BatchNormalization()(features,training=training)
  y = tf.keras.layers.Dense(1)(x)
  predictions = {
      "prob": y,
  }
  
  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  loss = tf.reduce_mean(tf.losses.mean_squared_error(labels,y))

  if mode == tf.estimator.ModeKeys.TRAIN:
      optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
     
      update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
      with tf.control_dependencies(update_ops):
        train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())

      return tf.estimator.EstimatorSpec(
          mode=tf.estimator.ModeKeys.TRAIN,
          loss=loss,
          train_op=train_op)
    
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss)
  
estimator = tf.estimator.Estimator(
    model_fn=model_fn)

input_fn = lambda:(tf.constant([[0], [1], [2], [3]], dtype=tf.float32),tf.constant([[0], [-1], [-2], [-3]], dtype=tf.float32))

estimator.train(input_fn,steps=5000)
result = estimator.evaluate(input_fn,steps=4)

tf.keras.layers.BatchNormaliztion　＆　model_to_estimator

In [0]:
import tensorflow as tf

inputs = tf.keras.Input(shape=(1,))
x = tf.keras.layers.BatchNormalization()(inputs)
outputs = tf.keras.layers.Dense(1)(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam',
              loss='mse',
              metrics=['accuracy'])

estimator = tf.keras.estimator.model_to_estimator(model)

input_fn = lambda:(tf.constant([[0], [1], [2], [3]], dtype=tf.float32),tf.constant([[0], [-1], [-2], [-3]], dtype=tf.float32))

estimator.train(input_fn,steps=5000)
result = estimator.evaluate(input_fn,steps=4)
print(result)

Traditional style sample

In [6]:
import tensorflow as tf
import numpy as np

tf.reset_default_graph()
inputs = tf.placeholder(shape=[None,1], dtype=tf.float32)
labels = tf.placeholder(shape=[None,1], dtype=tf.float32)
training = tf.placeholder(shape=[], dtype=tf.bool)

with tf.variable_scope('batch_normalization_test'):
  with tf.name_scope('for_train'):
    BN1 =  tf.layers.BatchNormalization(name="bn1")
    x = BN1(inputs,training =training)
    x = tf.layers.Dense(units=1,name="dense1")(x)
    BN2 =  tf.layers.BatchNormalization(name="bn2")
    x = BN2(x,training =training)
    output1 = tf.layers.Dense(units=1,name="dense2")(x)

with tf.variable_scope('batch_normalization_test', reuse=True):
  with tf.name_scope('for_test'):
    BN1 =  tf.layers.BatchNormalization(name="bn1")
    x = BN1(inputs,training =training)
    x = tf.layers.Dense(units=1,name="dense1")(x)
    BN2 =  tf.layers.BatchNormalization(name="bn2")
    x = BN2(x,training =training)
    output2 = tf.layers.Dense(units=1,name="dense2")(x)

loss = tf.reduce_mean(tf.losses.mean_squared_error(labels,output1))
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
grads = optimizer.compute_gradients(loss)
with tf.control_dependencies(update_ops):
  train_op = optimizer.apply_gradients(grads)

  sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

train_x = np.array([[-3.],[-2.],[-1.],[0.]])
train_y = np.array([[0.], [1.], [2.], [3.]])

for _ in range(5000):
  sess.run((loss,train_op,output1,output2),{inputs:train_x,labels:train_y,training:True})
#  print(sess.run(BN1.weights))
#  print(sess.run(BN2.weights))

print(sess.run((loss,output1,output2),{inputs:train_x,labels:train_y,training:True}))
print(sess.run((output1),{inputs:train_x,training:False}))
print(sess.run((output2),{inputs:train_x,training:False}))

(1.43538514e-11, array([[6.2584877e-06],
       [1.0000033e+00],
       [2.0000005e+00],
       [2.9999974e+00]], dtype=float32), array([[6.2584877e-06],
       [1.0000033e+00],
       [2.0000005e+00],
       [2.9999974e+00]], dtype=float32))
[[-1.6331673e-05]
 [ 9.9998498e-01]
 [ 1.9999862e+00]
 [ 2.9999876e+00]]
[[-1.6331673e-05]
 [ 9.9998498e-01]
 [ 1.9999862e+00]
 [ 2.9999876e+00]]
